*AKASH AND PRADHYUMNA KAGGLE CHALLENGE 2*

In [ ]:
!pip install pycaret

In [ ]:
!pip install "schemdraw<0.16"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing required packages

import pandas as pd
from sklearn.metrics import mean_squared_error
from pycaret.regression import *
import numpy as np
import xgboost as xgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# We create models based on Political Leanings. 
# We choose 3 Political leanings : Republican, Mixed, Democrat
# These are political leanings of each state

state_political_leanings = {
    "AL": "Republican",
    "AK": "Republican",
    "AZ": "Mixed",
    "AR": "Republican",
    "CA": "Democrat",
    "CO": "Democrat",
    "CT": "Democrat",
    "DE": "Democrat",
    "FL": "Mixed",
    "GA": "Mixed",
    "HI": "Democrat",
    "ID": "Republican",
    "IL": "Democrat",
    "IN": "Republican",
    "IA": "Mixed",
    "KS": "Republican",
    "KY": "Republican",
    "LA": "Republican",
    "ME": "Mixed",
    "MD": "Democrat",
    "MA": "Democrat",
    "MI": "Mixed",
    "MN": "Mixed",
    "MS": "Republican",
    "MO": "Republican",
    "MT": "Republican",
    "NE": "Republican",
    "NV": "Mixed",
    "NH": "Mixed",
    "NJ": "Democrat",
    "NM": "Democrat",
    "NY": "Democrat",
    "NC": "Mixed",
    "ND": "Republican",
    "OH": "Mixed",
    "OK": "Republican",
    "OR": "Democrat",
    "PA": "Mixed",
    "RI": "Democrat",
    "SC": "Republican",
    "SD": "Republican",
    "TN": "Republican",
    "TX": "Mixed",
    "UT": "Republican",
    "VT": "Democrat",
    "VA": "Democrat",
    "WA": "Democrat",
    "WV": "Republican",
    "WI": "Mixed",
    "WY": "Republican",
    "DC":'Mixed'
}

In [ ]:
def fun(state):
  return state_political_leanings[state]

In [ ]:
# Reading the Datasets
training_data = pd.read_csv("/content/drive/MyDrive/Kaggle 2/training_data.csv")

In [ ]:
training_data

,general_sector,city,zip_code,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,...,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,Retired,MISSOULA,59802.0,Retired,MT,Individual,1.0,4,4,0,...,0.5,0.000000,0.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0
1,Retired,CLEVELAND HTS,44118.0,Retired,OH,Individual,0.5,2,2,0,...,0.0,0.216202,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0
2,Public Sector Unions,NEWARK,NaN,Teachers unions,NJ,Non-Individual,1.0,4,4,0,...,0.0,3.257082,0.0,1.0,1.0,0.0,0.0,-2.257082,0.0,0.0
3,Retired,BOZEMAN,59715.0,Retired,MT,Individual,1.0,4,4,0,...,0.6,0.200000,0.0,1.0,0.5,0.5,0.0,0.2,0.0,0.0
4,No Employer Listed or Found,ORANGE PARK,32003.0,No employer listed or discovered,FL,Individual,0.0,4,4,0,...,1.0,0.000000,0.5,1.0,0.0,0.5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172843,Savings & Loans,FAIRFIELD,6825.0,Savings banks & savings and loans,CT,Individual,1.0,2,0,2,...,0.0,0.000000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
172844,Uncoded,AUSTIN,78733.0,Uncoded,DE,Individual,1.0,2,2,0,...,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
172845,Uncoded,PATERSON,7442.0,Uncoded,NJ,Non-Individual,1.0,2,0,2,...,0.0,0.000000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
172846,Uncoded,NEW ORLEANS,70122.0,Uncoded,LA,Individual,0.0,2,0,2,...,1.0,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
income_data = pd.read_csv("/content/final_data_income.csv")

In [ ]:
# We use income of each zip code of city. Indicating each city's income which may help us in identifying the Contributions from each state.

In [ ]:
income_data

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,general_sector,city,zip_code,Income,specific_sector,state
0,0,2438,2438,Retired,MISSOULA,59802.0,778210.0,Retired,MT
1,1,1,1,Retired,CLEVELAND HTS,44118.0,778210.0,Retired,OH
2,2,2,2,Public Sector Unions,NEWARK,43215.0,778210.0,Teachers unions,NJ
3,3,3,3,Retired,BOZEMAN,59715.0,778210.0,Retired,MT
4,4,4,4,No Employer Listed or Found,ORANGE PARK,32003.0,778210.0,No employer listed or discovered,FL
...,...,...,...,...,...,...,...,...,...
288075,288075,288075,115227,Lawyers & Lobbyists,RALEIGH,27612.0,70.0,Attorneys & law firms,US
288076,288076,288076,115228,Uncoded,ELEPHANT BUTTE,87935.0,30.0,Uncoded,NM
288077,288077,288077,115229,Uncoded,BOCA RATON,33496.0,60.0,Uncoded,NJ
288078,288078,288078,115230,Trucking,HARTFORD,6105.0,2438.0,Trucking,CT


In [ ]:
income_data = income_data.drop_duplicates(subset = 'zip_code')

In [ ]:
income_data

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,general_sector,city,zip_code,Income,specific_sector,state
0,0,2438,2438,Retired,MISSOULA,59802.0,778210.0,Retired,MT
1,1,1,1,Retired,CLEVELAND HTS,44118.0,778210.0,Retired,OH
2,2,2,2,Public Sector Unions,NEWARK,43215.0,778210.0,Teachers unions,NJ
3,3,3,3,Retired,BOZEMAN,59715.0,778210.0,Retired,MT
4,4,4,4,No Employer Listed or Found,ORANGE PARK,32003.0,778210.0,No employer listed or discovered,FL
...,...,...,...,...,...,...,...,...,...
287907,287907,287907,115059,Uncoded,SHREVEPORT,71136.0,380.0,Uncoded,LA
287924,287924,287924,115076,Retired,KALAMAZOO,49048.0,350.0,Retired,US
287932,287932,287932,115084,Party Committees,SALEM,271.0,460.0,Republican Party committees,NC
288032,288032,288032,115184,Lawyers & Lobbyists,HOUSTON,77045.0,30.0,Attorneys & law firms,TX


In [ ]:
income_data['Income'] = income_data['Income'].astype(float)

In [ ]:
min_value = income_data['Income'].min()
max_value = income_data['Income'].max()

In [ ]:
print(income_data['zip_code'].max())

99999.0


In [ ]:
income_data['normalized_Income'] = (income_data['Income'] - min_value) / (max_value - min_value)

In [ ]:
income_data['normalized_Income'] = income_data['normalized_Income'].apply(lambda x: round(x,5))

In [ ]:
income_data = dict(zip(income_data.zip_code, income_data.normalized_Income))

In [ ]:
income_data

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/Kaggle 2/test_data.csv")
#test_data = test_data.drop(columns = 'winner_ratio')

In [ ]:
test_data.shape

(115232, 42)

In [ ]:
#spliting data into train test split as given orignally
training_data['train_test_split'] = 1
test_data['train_test_split'] = 0
data = pd.concat([training_data,test_data])

In [ ]:
data['general_sector'].unique()

In [ ]:
data['contributor_type'] = data['contributor_type'].replace("Other","Another")

In [ ]:
data['contributor_type'].value_counts()

Individual        242733
Non-Individual     45224
Another              123
Name: contributor_type, dtype: int64

In [ ]:
avg = sum(list(income_data.values()))/len(list(income_data.values()))

In [ ]:
avg

0.009350900665928042

In [ ]:
len(data['zip_code'].unique())

18020

In [ ]:
import math

In [ ]:
def zip2income(zip):
  #print(zip,type(zip))
  if math.isnan(zip) or zip == 0.0:
    return avg
  else:
    return income_data[zip]

In [ ]:
data['zip_code']

0         59802.0
1         44118.0
2             NaN
3         59715.0
4         32003.0
           ...   
115227    27612.0
115228    87935.0
115229    33496.0
115230     6105.0
115231     6108.0
Name: zip_code, Length: 288080, dtype: float64

In [ ]:
data['Income'] = data['zip_code'].apply(zip2income)

In [ ]:
data['Income']

0         0.405950
1         0.405950
2         0.009351
3         0.405950
4         0.405950
            ...   
115227    0.000230
115228    0.000050
115229    0.001270
115230    0.405950
115231    0.000050
Name: Income, Length: 288080, dtype: float64

In [ ]:
data.state.value_counts().to_dict()

In [ ]:
data = data.drop(columns = ['zip_code'])

In [ ]:
# List of features that we inted to be keep

#columns = ['contribution_count','politician_count','contribution_sum_2010_usd','politician_challenger_count']
colums_to_keep = ['general_sector','state','winner_ratio','contributor_type','candidacy_democratic_ratio','candidacy_republican_ratio',
                  'contribution_democratic_count_ratio','contribution_republican_count_ratio','governor_contribution_ratio',
                  'house_and_assembly_contribution_ratio','politician_challenger_ratio','politician_democratic_ratio','politician_incumbency_ratio',
                  'politician_open_pos_ratio','politician_republican_ratio','senate_contribution_ratio','train_test_split']

In [ ]:
#data = data[colums_to_keep]

# Preprocessing to read the features

data.replace([np.inf, -np.inf], 0, inplace=True)
data['senate_contribution_ratio']   = data['senate_contribution_ratio'].replace(['#NAME?'], None)
data['governor_contribution_ratio'] = data['governor_contribution_ratio'].replace(['#NAME?'], None)

data['senate_contribution_ratio']   = data['senate_contribution_ratio'].astype(float)
data['governor_contribution_ratio'] = data['governor_contribution_ratio'].astype(float)

In [ ]:
data['us_senate_contribution_ratio'] = data['us_senate_contribution_ratio'].replace(['#NAME?'], None)
data['us_senate_contribution_ratio'] = data['us_senate_contribution_ratio'].astype(float)

In [ ]:
data = data.drop(columns = ["city","specific_sector"])

In [ ]:
#general_sector =  pd.get_dummies(data['general_sector'])
contributor_type =  pd.get_dummies(data['contributor_type'])

In [ ]:
general_sector

In [ ]:
data  = pd.concat([data,contributor_type],axis =1)
data = data.drop(columns = ['general_sector','contributor_type'])

In [ ]:
data

,state,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,train_test_split,Income,Another,Individual,Non-Individual
0,MT,1.0,4,4,0,6,6,0,0,4,...,0.500000,0.000000,0.000000,0.0,0.0,1,0.405950,0,1,0
1,OH,0.5,2,2,0,2,2,0,1,2,...,0.000000,0.000000,0.000000,0.0,0.0,1,0.405950,0,1,0
2,NJ,1.0,4,4,0,4,4,0,0,4,...,0.000000,0.000000,-2.257082,0.0,0.0,1,0.009351,0,0,1
3,MT,1.0,4,4,0,4,4,0,0,4,...,0.500000,0.000000,0.200000,0.0,0.0,1,0.405950,0,1,0
4,FL,0.0,4,4,0,6,6,0,2,4,...,0.500000,0.000000,0.000000,0.0,0.0,1,0.405950,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115227,US,NaN,2,2,0,2,2,0,0,2,...,0.000000,0.000000,0.000000,1.0,0.0,0,0.000230,0,1,0
115228,NM,NaN,2,0,2,2,0,2,0,2,...,0.000000,1.000000,0.000000,0.0,0.0,0,0.000050,0,1,0
115229,NJ,NaN,2,0,2,2,0,2,0,2,...,0.000000,1.000000,0.000000,0.0,0.0,0,0.001270,0,1,0
115230,CT,NaN,135,95,40,152,105,47,4,99,...,0.131313,0.343434,0.417607,0.0,0.0,0,0.405950,0,0,1


In [ ]:
data.dtypes

state                                             object
winner_ratio                                     float64
candidacy_count                                    int64
candidacy_democratic_count                         int64
candidacy_republican_count                         int64
contribution_count                                 int64
contribution_democratic_count                      int64
contribution_republican_count                      int64
politician_challenger_count                        int64
politician_count                                   int64
politician_democratic_count                        int64
politician_incumbency_count                        int64
politician_open_pos_count                          int64
politician_republican_count                        int64
contribution_democratic_sum_2010_usd             float64
contribution_republican_sum_2010_usd             float64
contribution_sum_2010_usd                        float64
governor_contributions_sum_2010

In [ ]:
float64_cols = list(data.select_dtypes(include='float64'))

# The same code again calling the columns
data[float64_cols] = data[float64_cols].astype('float32')

float64_cols = list(data.select_dtypes(include='int64'))

# The same code again calling the columns
data[float64_cols] = data[float64_cols].astype('int32')

float64_cols = list(data.select_dtypes(include='uint8'))

# The same code again calling the columns
data[float64_cols] = data[float64_cols].astype('int8')

In [ ]:
float64_cols

In [ ]:
training_data = data[data['train_test_split'] == 1]
test_data = data[data['train_test_split'] == 0]

training_data = training_data.drop(columns = 'train_test_split')
test_data = test_data.drop(columns = 'train_test_split')

In [ ]:
training_data['state'].value_counts()

In [ ]:
# Segregating data wof state which are US and Not US state. Indicating Federal and State level inclination.

training_data_1 = training_data[training_data['state'] == "US"]
#trarget_y_1 = training_data_1['winner_ratio']
training_data_1 = training_data_1.drop(columns =['state'])

In [ ]:
training_data_2 = training_data[training_data['state'] != "US"]
training_data_2 = training_data_2.dropna()
#trarget_y_2 = training_data_2['winner_ratio']
training_data_2['state_political_leanings'] = training_data_2['state'].apply(fun)
#training_data_2 = training_data_2.drop(columns = ['state'])

In [ ]:
leanings = pd.get_dummies(training_data_2['state_political_leanings'])

In [ ]:
training_data_2 = pd.concat([training_data_2,leanings],axis = 1)

In [ ]:
# Droping the y variable
test_data  = test_data.drop(columns = "winner_ratio")

In [ ]:
# Creating two models based on State.
# One from those which are US, Second being those which are not US states.

test_data_1 = test_data[test_data['state'] == "US"]
test_data_2 = test_data[test_data['state'] != "US"]
test_data_2['state_political_leanings'] = test_data_2['state'].apply(fun)

In [ ]:
training_data_2.state_political_leanings.value_counts()

Democrat      67185
Mixed         52841
Republican    14156
Name: state_political_leanings, dtype: int64

In [ ]:
test_data_2

,zip_code,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual,state_political_leanings
0,27948.0,NC,6,6,0,10,10,0,0,4,...,0.500000,0.000000,0.000000,1.000000,0.0,0.0,0,1,0,Mixed
1,60659.0,NJ,2,0,2,2,0,2,0,2,...,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0,1,0,Democrat
2,50023.0,IA,2,0,2,2,0,2,0,2,...,0.000000,1.000000,1.000000,0.000000,0.0,0.0,0,1,0,Mixed
3,33182.0,FL,2,0,2,6,0,6,0,2,...,0.000000,1.000000,1.000000,0.000000,0.0,0.0,0,1,0,Mixed
4,8075.0,NJ,2,0,2,2,0,2,0,2,...,1.000000,0.000000,1.000000,1.000000,0.0,0.0,0,0,1,Democrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115226,34491.0,FL,2,2,0,3,3,0,0,2,...,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0,1,0,Mixed
115228,87935.0,NM,2,0,2,2,0,2,0,2,...,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0,1,0,Democrat
115229,33496.0,NJ,2,0,2,2,0,2,0,2,...,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0,1,0,Democrat
115230,6105.0,CT,135,95,40,152,105,47,4,99,...,0.828283,0.131313,0.343434,0.417607,0.0,0.0,0,0,1,Democrat


In [ ]:
test_data_1

,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,...,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Income,Another,Individual,Non-Individual
15,US,6,6,0,7,7,0,0,4,4,...,1.000000,0.000000,0.000000,0.0,0.847994,0.152006,0.00037,0,1,0
17,US,2,0,2,2,0,2,0,2,0,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.00029,0,1,0
23,US,18,14,4,23,17,6,2,17,14,...,0.647059,0.235294,0.176471,0.0,0.788515,0.211485,0.40595,0,1,0
31,US,2,0,2,2,0,2,0,2,0,...,0.000000,1.000000,1.000000,0.0,0.648160,0.351840,0.40595,0,1,0
36,US,2,0,2,2,0,2,0,2,0,...,0.000000,1.000000,1.000000,0.0,1.000000,0.000000,0.40595,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115203,US,5,5,0,10,10,0,0,5,5,...,0.400000,0.600000,0.000000,0.0,0.271130,0.728870,0.00170,0,1,0
115213,US,2,2,0,2,2,0,0,2,2,...,1.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.00035,0,1,0
115214,US,6,6,0,10,10,0,4,6,6,...,0.000000,0.333333,0.000000,0.0,1.000000,0.000000,0.00002,0,1,0
115225,US,8,8,0,25,25,0,4,7,7,...,0.000000,0.428571,0.000000,0.0,0.712832,0.287168,0.00013,0,1,0


In [ ]:
training_data_1 

,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,...,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual
6,1.000000,7,7,0,14,14,0,0,4,4,...,1.000000,1.000000,0.000000,0.000000,0.0,0.922284,0.077716,0,1,0
8,0.750000,12,0,12,14,0,14,0,11,0,...,0.000000,0.727273,0.272727,1.000000,0.0,0.583355,0.416645,0,1,0
10,0.571429,7,1,6,8,1,7,1,7,1,...,0.142857,0.142857,0.714286,0.857143,0.0,0.269340,0.730660,0,1,0
12,1.000000,2,0,2,2,0,2,0,2,0,...,0.000000,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,0,1,0
15,0.666667,24,0,24,45,0,45,8,18,0,...,0.000000,0.500000,0.055556,1.000000,0.0,0.671888,0.328112,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172831,0.000000,3,0,3,5,0,5,3,3,0,...,0.000000,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0,1,0
172833,1.000000,4,4,0,6,6,0,0,4,4,...,1.000000,0.000000,1.000000,0.000000,0.0,0.591819,0.408181,0,1,0
172837,0.000000,2,2,0,5,5,0,0,2,2,...,1.000000,0.500000,0.500000,0.000000,0.0,0.000000,1.000000,0,1,0
172838,0.750000,16,16,0,29,29,0,1,11,11,...,1.000000,0.454545,0.454545,0.000000,0.0,0.828301,0.171699,0,1,0


In [ ]:
training_data_1 = training_data_1.drop(columns = 'zip_code')
training_data_2 = training_data_2.drop(columns = 'zip_code')

##FED MODEL

In [ ]:
# Running Pycaret model to get the best model.

setup(data=training_data_1, target='winner_ratio', session_id=42) #use_gpu = True,remove_multicollinearity = True,remove_outliers = True, normalize = True,pca = True,feature_selection=True,imputation_type = "simple")

,Description,Value
0,Session id,42
1,Target,winner_ratio
2,Target type,Regression
3,Original data shape,"(34622, 38)"
4,Transformed data shape,"(34622, 38)"
5,Transformed train set shape,"(24235, 38)"
6,Transformed test set shape,"(10387, 38)"
7,Numeric features,37
8,Rows with missing values,0.2%
9,Preprocess,True


In [ ]:
training_data_1

,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,...,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual
6,1.000000,7,7,0,14,14,0,0,4,4,...,1.000000,1.000000,0.000000,0.000000,0.0,0.922284,0.077716,0,1,0
8,0.750000,12,0,12,14,0,14,0,11,0,...,0.000000,0.727273,0.272727,1.000000,0.0,0.583355,0.416645,0,1,0
10,0.571429,7,1,6,8,1,7,1,7,1,...,0.142857,0.142857,0.714286,0.857143,0.0,0.269340,0.730660,0,1,0
12,1.000000,2,0,2,2,0,2,0,2,0,...,0.000000,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,0,1,0
15,0.666667,24,0,24,45,0,45,8,18,0,...,0.000000,0.500000,0.055556,1.000000,0.0,0.671888,0.328112,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172831,0.000000,3,0,3,5,0,5,3,3,0,...,0.000000,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0,1,0
172833,1.000000,4,4,0,6,6,0,0,4,4,...,1.000000,0.000000,1.000000,0.000000,0.0,0.591819,0.408181,0,1,0
172837,0.000000,2,2,0,5,5,0,0,2,2,...,1.000000,0.500000,0.500000,0.000000,0.0,0.000000,1.000000,0,1,0
172838,0.750000,16,16,0,29,29,0,1,11,11,...,1.000000,0.454545,0.454545,0.000000,0.0,0.828301,0.171699,0,1,0


In [ ]:
#Comparing different models and seeing which is the best fit

best_model_fed = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.2068,0.0757,0.2751,0.4368,0.1893,0.2860,0.9980
xgboost,Extreme Gradient Boosting,0.2064,0.0770,0.2775,0.4267,0.1906,0.2898,6.0500
gbr,Gradient Boosting Regressor,0.2171,0.0800,0.2828,0.4048,0.1947,0.2941,6.5640
rf,Random Forest Regressor,0.2003,0.0812,0.2850,0.3955,0.1947,0.2926,19.3440
et,Extra Trees Regressor,0.2035,0.0918,0.3030,0.3167,0.2066,0.3054,9.0040
ridge,Ridge Regression,0.2365,0.0919,0.3031,0.3162,0.2087,0.3036,0.2020
br,Bayesian Ridge,0.2366,0.0919,0.3031,0.3160,0.2087,0.3037,0.2260
ada,AdaBoost Regressor,0.2533,0.0932,0.3053,0.3061,0.2107,0.3165,0.8040
knn,K Neighbors Regressor,0.2682,0.1203,0.3468,0.1048,0.2355,0.3796,0.7940
dt,Decision Tree Regressor,0.2326,0.1277,0.3573,0.0493,0.2433,0.3741,0.4760


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
# Using light gradient boosting
model_fed = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2123,0.0803,0.2833,0.4085,0.1964,0.3013
1,0.2055,0.0739,0.2719,0.4395,0.1863,0.2917
2,0.2038,0.0734,0.2710,0.4585,0.1890,0.2818
3,0.2097,0.0785,0.2803,0.4120,0.1925,0.2845
4,0.2049,0.0735,0.2711,0.4255,0.1855,0.2821
5,0.2017,0.0722,0.2687,0.4638,0.1848,0.2725
6,0.2018,0.0722,0.2688,0.4701,0.1843,0.2750
7,0.2115,0.0781,0.2794,0.4078,0.1905,0.2917
8,0.2087,0.0768,0.2772,0.4364,0.1903,0.2901


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pred = predict_model(model_fed,data = test_data_1)

In [ ]:
test_data_1


,zip_code,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual
15,2806.0,US,6,6,0,7,7,0,0,4,...,1.000000,1.000000,0.000000,0.000000,0.0,0.847994,0.152006,0,1,0
17,2468.0,US,2,0,2,2,0,2,0,2,...,0.000000,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,0,1,0
23,22101.0,US,18,14,4,23,17,6,2,17,...,0.823529,0.647059,0.235294,0.176471,0.0,0.788515,0.211485,0,1,0
31,4105.0,US,2,0,2,2,0,2,0,2,...,0.000000,0.000000,1.000000,1.000000,0.0,0.648160,0.351840,0,1,0
36,7006.0,US,2,0,2,2,0,2,0,2,...,0.000000,0.000000,1.000000,1.000000,0.0,1.000000,0.000000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115203,11215.0,US,5,5,0,10,10,0,0,5,...,1.000000,0.400000,0.600000,0.000000,0.0,0.271130,0.728870,0,1,0
115213,32904.0,US,2,2,0,2,2,0,0,2,...,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,1.000000,0,1,0
115214,33401.0,US,6,6,0,10,10,0,4,6,...,1.000000,0.000000,0.333333,0.000000,0.0,1.000000,0.000000,0,1,0
115225,20001.0,US,8,8,0,25,25,0,4,7,...,1.000000,0.000000,0.428571,0.000000,0.0,0.712832,0.287168,0,1,0


In [ ]:
pred

,zip_code,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual,prediction_label
15,2806.0,US,6,6,0,7,7,0,0,4,...,1.000000,0.000000,0.000000,0.0,0.847994,0.152006,0,1,0,0.862611
17,2468.0,US,2,0,2,2,0,2,0,2,...,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,0,1,0,0.921089
23,22101.0,US,18,14,4,23,17,6,2,17,...,0.647059,0.235294,0.176471,0.0,0.788515,0.211485,0,1,0,0.721493
31,4105.0,US,2,0,2,2,0,2,0,2,...,0.000000,1.000000,1.000000,0.0,0.648160,0.351840,0,1,0,0.352974
36,7006.0,US,2,0,2,2,0,2,0,2,...,0.000000,1.000000,1.000000,0.0,1.000000,0.000000,0,1,0,0.367910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115203,11215.0,US,5,5,0,10,10,0,0,5,...,0.400000,0.600000,0.000000,0.0,0.271130,0.728870,0,1,0,0.561636
115213,32904.0,US,2,2,0,2,2,0,0,2,...,1.000000,0.000000,0.000000,0.0,0.000000,1.000000,0,1,0,0.858528
115214,33401.0,US,6,6,0,10,10,0,4,6,...,0.000000,0.333333,0.000000,0.0,1.000000,0.000000,0,1,0,0.321576
115225,20001.0,US,8,8,0,25,25,0,4,7,...,0.000000,0.428571,0.000000,0.0,0.712832,0.287168,0,1,0,0.296366


##SINGLE STATE LEVEL MODEL

In [ ]:
setup(data=training_data_2, target=trarget_y_2, session_id=42) #use_gpu = True,remove_multicollinearity = True,remove_outliers = True, normalize = True,pca = True,feature_selection=True,imputation_type = "simple")

,Description,Value
0,Session id,42
1,Target,winner_ratio
2,Target type,Regression
3,Original data shape,"(138085, 36)"
4,Transformed data shape,"(138085, 38)"
5,Transformed train set shape,"(96659, 38)"
6,Transformed test set shape,"(41426, 38)"
7,Numeric features,34
8,Categorical features,1
9,Preprocess,True


In [ ]:
best_model_state = compare_models()

In [ ]:
model_state = create_model(best_model_state)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2385,0.0996,0.3155,0.4399,0.2191,0.2688
1,0.2425,0.1012,0.3182,0.4253,0.2205,0.2798
2,0.2398,0.1000,0.3162,0.4250,0.2197,0.2780
3,0.2465,0.1035,0.3217,0.4280,0.2238,0.2809
4,0.2448,0.1035,0.3217,0.4169,0.2229,0.2789
5,0.2369,0.0981,0.3132,0.4463,0.2170,0.2735
6,0.2419,0.1016,0.3187,0.4279,0.2214,0.2722
7,0.2402,0.1004,0.3168,0.4236,0.2198,0.2727
8,0.2442,0.1024,0.3200,0.4256,0.2222,0.2820


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
test_data_2 = test_data_2.drop(columns = "winner_ratio")

In [ ]:
test_data_2 = test_data_2.drop(columns = "winner_ratio")

In [ ]:
pred2 = predict_model(model,data = test_data_2)

In [ ]:
pred2 = pred2['prediction_label']

In [ ]:
pred_final = pd.concat([pred,pred2])

In [ ]:
pred_final

15        0.840003
17        0.961667
23        0.733966
31        0.645833
36        0.306667
            ...   
115226    0.320254
115228    0.883270
115229    0.981826
115230    0.888507
115231    0.597652
Name: prediction_label, Length: 115232, dtype: float64

In [ ]:
pred_final = pred_final.sort_index()

In [ ]:
test_data

,general_sector,city,zip_code,specific_sector,state,contributor_type,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,...,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,Food & Beverage,KILL DEVIL HILLS,27948.0,Restaurants & drinking establishments,NC,Individual,6,6,0,10,...,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,1.000000,0.0,0.0
1,Uncoded,CHICAGO,60659.0,Uncoded,NJ,Individual,2,0,2,2,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0
2,Lawyers & Lobbyists,ANKENY,50023.0,Lobbyists & public relations,IA,Individual,2,0,2,2,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.0
3,Retired,MIAMI,33182.0,Retired,FL,Individual,2,0,2,6,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.0
4,Uncoded,RIVERSIDE,8075.0,Uncoded,NJ,Non-Individual,2,0,2,2,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115227,Lawyers & Lobbyists,RALEIGH,27612.0,Attorneys & law firms,US,Individual,2,2,0,2,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.0,0.0
115228,Uncoded,ELEPHANT BUTTE,87935.0,Uncoded,NM,Individual,2,0,2,2,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0
115229,Uncoded,BOCA RATON,33496.0,Uncoded,NJ,Individual,2,0,2,2,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.0
115230,Trucking,HARTFORD,6105.0,Trucking,CT,Non-Individual,135,95,40,152,...,0.019563,0.525758,0.040404,0.656566,0.828283,0.131313,0.343434,0.417607,0.0,0.0


In [ ]:
pred_final.to_csv("sub.csv")

In [ ]:
reg = setup(data = training_data_2,
            target = trarget_y_2,
            silent = True)

TypeError: ignored

## REP | DEM | MIXED Models at state level 

In [ ]:
# We choose 3 Political leanings : Republican, Mixed, Democrat

In [ ]:
training_data_2

,state,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual,state_political_leanings,Democrat,Mixed,Republican
0,MT,1.0,4,4,0,6,6,0,0,4,...,0.0,0.0,0.0,0,1,0,Republican,0,0,1
1,OH,0.5,2,2,0,2,2,0,1,2,...,0.0,0.0,0.0,0,1,0,Mixed,0,1,0
3,MT,1.0,4,4,0,4,4,0,0,4,...,0.2,0.0,0.0,0,1,0,Republican,0,0,1
4,FL,0.0,4,4,0,6,6,0,2,4,...,0.0,0.0,0.0,0,1,0,Mixed,0,1,0
5,NC,1.0,2,2,0,4,4,0,0,2,...,0.0,0.0,0.0,0,1,0,Mixed,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172843,CT,1.0,2,0,2,2,0,2,0,2,...,1.0,0.0,0.0,0,1,0,Democrat,1,0,0
172844,DE,1.0,2,2,0,4,4,0,2,2,...,0.0,0.0,0.0,0,1,0,Democrat,1,0,0
172845,NJ,1.0,2,0,2,2,0,2,0,2,...,1.0,0.0,0.0,0,0,1,Democrat,1,0,0
172846,LA,0.0,2,0,2,2,0,2,0,2,...,0.0,0.0,0.0,0,1,0,Republican,0,0,1


In [ ]:
training_data_dem = training_data_2[training_data_2['state_political_leanings'] == "Democrat"]

training_data_rep = training_data_2[training_data_2['state_political_leanings'] == "Republican"]

training_data_mix = training_data_2[training_data_2['state_political_leanings'] == "Mixed"]


In [ ]:
training_data_dem = training_data_dem.drop(columns = "state_political_leanings")
training_data_rep = training_data_rep.drop(columns = "state_political_leanings")
training_data_mix = training_data_mix.drop(columns = "state_political_leanings")

In [ ]:
test_data_2[test_data_2['state_political_leanings'] == None]

,zip_code,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual,state_political_leanings


In [ ]:
test_data_2['state_political_leanings'].isnull().values.any()

False

In [ ]:
leanins = pd.get_dummies(test_data_2['state_political_leanings'])
test_data_2 = pd.concat([test_data_2,leanins],axis = 1)


In [ ]:
test_dem = test_data_2[test_data_2['state_political_leanings'] == 'Democrat']
test_rep = test_data_2[test_data_2['state_political_leanings'] == 'Republican']
test_mix = test_data_2[test_data_2['state_political_leanings'] == 'Mixed']

In [ ]:
test_dem = test_dem.drop(columns = "state_political_leanings")
test_rep = test_rep.drop(columns = "state_political_leanings")
test_mix = test_mix.drop(columns = "state_political_leanings")

###DEM

In [ ]:
setup(data=training_data_dem, target='winner_ratio', session_id=42) #use_gpu = True,remove_multicollinearity = True,remove_outliers = True, normalize = True,pca = True,feature_selection=True,imputation_type = "simple")

,Description,Value
0,Session id,42
1,Target,winner_ratio
2,Target type,Regression
3,Original data shape,"(67185, 42)"
4,Transformed data shape,"(67185, 56)"
5,Transformed train set shape,"(47029, 56)"
6,Transformed test set shape,"(20156, 56)"
7,Numeric features,40
8,Categorical features,1
9,Preprocess,True


In [ ]:
best_model_dem = compare_models(fold = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.1982,0.0818,0.2859,0.5648,0.1981,0.2255,13.5280
lightgbm,Light Gradient Boosting Machine,0.2026,0.0829,0.2880,0.5586,0.1998,0.2285,1.5900
rf,Random Forest Regressor,0.1713,0.0845,0.2908,0.5500,0.2002,0.2052,36.9280
et,Extra Trees Regressor,0.1691,0.0981,0.3131,0.4780,0.2151,0.2073,24.7960
gbr,Gradient Boosting Regressor,0.2390,0.1009,0.3177,0.4627,0.2205,0.2564,11.8780
ada,AdaBoost Regressor,0.2825,0.1146,0.3386,0.3897,0.2373,0.2908,1.7820
ridge,Ridge Regression,0.2612,0.1170,0.3421,0.3770,0.2380,0.2639,0.9520
br,Bayesian Ridge,0.2614,0.1170,0.3421,0.3770,0.2380,0.2638,0.5780
dt,Decision Tree Regressor,0.1853,0.1268,0.3561,0.3250,0.2440,0.2543,0.9000
knn,K Neighbors Regressor,0.2607,0.1363,0.3692,0.2742,0.2535,0.2993,3.5820


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
model_dem = create_model(best_model_dem)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1956,0.0788,0.2808,0.5818,0.1950,0.2231
1,0.2004,0.0838,0.2895,0.5584,0.2006,0.2198
2,0.1970,0.0818,0.2861,0.5626,0.1985,0.2273
3,0.1982,0.0815,0.2854,0.5675,0.1990,0.2270
4,0.1972,0.0819,0.2862,0.5620,0.1978,0.2292
5,0.1951,0.0791,0.2812,0.5789,0.1945,0.2244
6,0.1944,0.0785,0.2801,0.5744,0.1928,0.2246
7,0.1962,0.0822,0.2866,0.5625,0.1986,0.2183
8,0.1975,0.0820,0.2863,0.5580,0.1982,0.2253


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#test_dem = test_dem.drop(columns= "winner_ratio")

In [ ]:
test_dem.shape

(46693, 42)

In [ ]:
pred_dem = predict_model(model_dem,data = test_dem)

In [ ]:
pred_dem.shape

In [ ]:
pred_dem['prediction_label'].isnull().values.any()

False

###REP

In [ ]:
setup(data=training_data_rep, target='winner_ratio', session_id=42) #use_gpu = True,remove_multicollinearity = True,remove_outliers = True, normalize = True,pca = True,feature_selection=True,imputation_type = "simple")

,Description,Value
0,Session id,42
1,Target,winner_ratio
2,Target type,Regression
3,Original data shape,"(14156, 42)"
4,Transformed data shape,"(14156, 59)"
5,Transformed train set shape,"(9909, 59)"
6,Transformed test set shape,"(4247, 59)"
7,Numeric features,40
8,Categorical features,1
9,Preprocess,True


In [ ]:
best_model_rep = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.1924,0.0651,0.2550,0.4200,0.1737,0.2753,0.7320
rf,Random Forest Regressor,0.1839,0.0675,0.2598,0.3981,0.1758,0.2706,8.4360
xgboost,Extreme Gradient Boosting,0.1962,0.0699,0.2643,0.3766,0.1791,0.2852,3.5280
gbr,Gradient Boosting Regressor,0.2081,0.0710,0.2663,0.3674,0.1821,0.2846,3.3560
et,Extra Trees Regressor,0.1837,0.0765,0.2765,0.3179,0.1865,0.2780,5.6380
br,Bayesian Ridge,0.2299,0.0855,0.2923,0.2381,0.1994,0.3038,0.2780
ridge,Ridge Regression,0.2298,0.0855,0.2924,0.2377,0.1993,0.3041,0.2700
ada,AdaBoost Regressor,0.2366,0.0857,0.2928,0.2357,0.1997,0.3084,0.5560
dummy,Dummy Regressor,0.2805,0.1125,0.3353,-0.0026,0.2281,0.3673,0.4960
lasso,Lasso Regression,0.2803,0.1125,0.3354,-0.0030,0.2280,0.3672,0.2660


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
model_rep = create_model(best_model_rep)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1922,0.0633,0.2516,0.4191,0.1692,0.2835
1,0.1823,0.0593,0.2435,0.4478,0.1640,0.2648
2,0.1905,0.0641,0.2531,0.4356,0.1724,0.2828
3,0.1903,0.0632,0.2514,0.4474,0.1729,0.2633
4,0.1872,0.0630,0.2509,0.4459,0.1707,0.2675
5,0.1962,0.0656,0.2561,0.4243,0.1729,0.2783
6,0.1905,0.0669,0.2587,0.3977,0.1769,0.2886
7,0.1916,0.0630,0.2510,0.3932,0.1668,0.2839
8,0.2006,0.0676,0.2600,0.4349,0.1797,0.2698


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#evaluate_model(model_rep)

In [ ]:
pred_rep = predict_model(model_rep,data = test_rep)

In [ ]:
test_rep.shape

(9800, 39)

In [ ]:
pred_rep

,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,...,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual,state_political_leanings,prediction_label
10,LA,2,0,2,2,0,2,0,2,0,...,1.000000,1.000000,0.000000,0.0,0.0,0,1,0,Republican,0.553740
13,MO,9,8,1,9,8,1,3,9,8,...,0.444444,0.111111,0.424199,0.0,0.0,0,1,0,Republican,0.463917
39,LA,2,0,2,3,0,3,0,2,0,...,0.500000,1.000000,0.000000,0.0,0.0,0,0,1,Republican,0.863629
47,LA,4,1,3,6,1,5,1,3,1,...,0.000000,0.666667,0.000000,0.0,0.0,0,1,0,Republican,0.493616
53,LA,9,4,5,10,4,6,1,9,4,...,0.555556,0.555556,0.060830,0.0,0.0,0,1,0,Republican,0.625816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115189,MT,3,3,0,6,6,0,0,2,2,...,0.000000,0.000000,0.000000,0.0,0.0,0,1,0,Republican,0.897975
115192,LA,2,2,0,2,2,0,0,2,2,...,1.000000,0.000000,0.000000,0.0,0.0,0,1,0,Republican,0.525106
115196,LA,3,0,3,4,0,4,0,3,0,...,1.000000,1.000000,0.000000,0.0,0.0,0,1,0,Republican,0.607968
115198,KY,13,1,12,19,1,18,3,9,1,...,0.333333,0.888889,0.311733,0.0,0.0,0,1,0,Republican,0.588266


In [ ]:
pred_rep['prediction_label'].isnull().values.any()

False

###Mix

In [ ]:
setup(data=training_data_mix, target='winner_ratio', session_id=42) #use_gpu = True,remove_multicollinearity = True,remove_outliers = True, normalize = True,pca = True,feature_selection=True,imputation_type = "simple")

,Description,Value
0,Session id,42
1,Target,winner_ratio
2,Target type,Regression
3,Original data shape,"(52841, 42)"
4,Transformed data shape,"(52841, 53)"
5,Transformed train set shape,"(36988, 53)"
6,Transformed test set shape,"(15853, 53)"
7,Numeric features,40
8,Categorical features,1
9,Preprocess,True


In [ ]:
best_model_mix = compare_models(fold=5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.2307,0.0929,0.3048,0.4725,0.2118,0.2820,1.1260
xgboost,Extreme Gradient Boosting,0.2290,0.0933,0.3054,0.4704,0.2118,0.2837,10.2020
rf,Random Forest Regressor,0.2063,0.0969,0.3112,0.4500,0.2145,0.2658,26.7840
et,Extra Trees Regressor,0.2037,0.1065,0.3264,0.3952,0.2244,0.2688,18.5080
gbr,Gradient Boosting Regressor,0.2639,0.1079,0.3285,0.3872,0.2286,0.3056,9.3660
br,Bayesian Ridge,0.2964,0.1312,0.3621,0.2553,0.2520,0.3242,0.8380
ridge,Ridge Regression,0.2963,0.1312,0.3622,0.2552,0.2520,0.3242,0.7660
ada,AdaBoost Regressor,0.3123,0.1332,0.3650,0.2436,0.2547,0.3493,1.5900
dt,Decision Tree Regressor,0.2265,0.1362,0.3691,0.2264,0.2536,0.3324,1.0820
knn,K Neighbors Regressor,0.2866,0.1462,0.3824,0.1698,0.2630,0.3638,1.9200


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
model_mix = create_model(best_model_mix)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2225,0.0875,0.2959,0.4923,0.2045,0.2786
1,0.2327,0.0958,0.3095,0.4610,0.2152,0.2839
2,0.2284,0.0909,0.3015,0.4839,0.2092,0.2773
3,0.2332,0.0937,0.3061,0.4716,0.2129,0.2845
4,0.2327,0.0951,0.3083,0.4631,0.2137,0.2777
5,0.2325,0.0937,0.3061,0.4721,0.2134,0.2859
6,0.2250,0.0890,0.2983,0.4925,0.2069,0.2750
7,0.2340,0.0959,0.3096,0.4611,0.2160,0.2864
8,0.2285,0.0909,0.3016,0.4798,0.2095,0.2801


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pred_mix = predict_model(model_mix,data = test_mix)

In [ ]:
test_mix.shape

(35854, 39)

In [ ]:
pred_mix

,zip_code,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,...,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Another,Individual,Non-Individual,Democrat,Mixed,Republican,prediction_label
0,27948.0,NC,6,6,0,10,10,0,0,4,...,1.000000,0.0,0.0,0,1,0,0,1,0,0.821492
2,50023.0,IA,2,0,2,2,0,2,0,2,...,0.000000,0.0,0.0,0,1,0,0,1,0,0.665302
3,33182.0,FL,2,0,2,6,0,6,0,2,...,0.000000,0.0,0.0,0,1,0,0,1,0,0.486103
5,28405.0,NC,6,6,0,12,12,0,0,6,...,0.000000,0.0,0.0,0,1,0,0,1,0,0.709465
7,20016.0,FL,2,2,0,2,2,0,0,2,...,0.000000,0.0,0.0,0,1,0,0,1,0,-0.003914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115210,44625.0,OH,4,0,4,5,0,5,0,3,...,0.000000,0.0,0.0,0,1,0,0,1,0,0.904426
115211,32082.0,FL,16,2,14,28,2,26,2,8,...,0.024807,0.0,0.0,0,1,0,0,1,0,0.589926
115217,28512.0,NC,6,4,2,8,6,2,0,6,...,0.000000,0.0,0.0,0,1,0,0,1,0,0.810303
115221,27607.0,NC,2,0,2,8,0,8,0,2,...,0.000000,0.0,0.0,0,1,0,0,1,0,0.499693


In [ ]:
pred_final = pd.concat([pred,pred_rep,pred_dem,pred_mix])

In [ ]:
pred_final = pred_final.sort_index()

In [ ]:
pred_final['prediction_label'].isnull().values.any()

False

In [ ]:
# Giving final predictions and seeing for test data.
# Predicting winner ratio and adding prediction label

In [ ]:
pred_final = pred_final['prediction_label']

In [ ]:
pred_final.to_csv("sub.csv")

In [ ]:
pred_final

,state,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,...,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio,Income,Another,Individual,Non-Individual,prediction_label,state_political_leanings
15,US,6,6,0,7,7,0,0,4,4,...,0.000000,0.000000,0.847994,0.152006,0.00037,0,1,0,0.862691,NaN
17,US,2,0,2,2,0,2,0,2,0,...,1.000000,0.000000,1.000000,0.000000,0.00029,0,1,0,0.928572,NaN
23,US,18,14,4,23,17,6,2,17,14,...,0.176471,0.000000,0.788515,0.211485,0.40595,0,1,0,0.721062,NaN
31,US,2,0,2,2,0,2,0,2,0,...,1.000000,0.000000,0.648160,0.351840,0.40595,0,1,0,0.334069,NaN
36,US,2,0,2,2,0,2,0,2,0,...,1.000000,0.000000,1.000000,0.000000,0.40595,0,1,0,0.358213,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115210,OH,4,0,4,5,0,5,0,3,0,...,1.000000,0.000000,0.000000,0.000000,0.00009,0,1,0,0.786478,Mixed
115211,FL,16,2,14,28,2,26,2,8,2,...,0.750000,0.024807,0.000000,0.000000,0.00013,0,1,0,0.665936,Mixed
115217,NC,6,4,2,8,6,2,0,6,4,...,0.333333,0.000000,0.000000,0.000000,0.00016,0,1,0,0.741887,Mixed
115221,NC,2,0,2,8,0,8,0,2,0,...,1.000000,0.000000,0.000000,0.000000,0.00003,0,1,0,0.553103,Mixed
